### 构建一个简单的 LLM 应用程序

在此快速入门中，我们将向您展示如何构建一个简单的 LLM 应用程序。

此应用程序将把文本从中文翻译成另一种语言。

这是一个相对简单的 LLM 应用程序 - 它只是一个 LLM 调用加上一些提示。

不过，这仍然是开始使用 LangChain 的好方法 - 只需一些提示和 LLM 调用就可以构建许多功能！


#### 概念
我们将介绍的概念包括：

1. 使用语言模型

2. 使用 PromptTemplates 和 OutputParsers

4. 使用 LangChain 链接 PromptTemplate + LLM + OutputParser

5. 使用 LangSmith 调试和跟踪您的应用程序

6. 使用 LangServe 部署您的应用程序

#### 本次将使用Jupyter Notebook

Jupyter Notebook
本指南（以及文档中的大多数其他指南）使用 Jupyter Notebook，并假设读者也使用 Jupyter Notebook。Jupyter Notebook 非常适合学习如何使用 LLM 系统，

因为很多时候可能会出错（意外输出、API 故障等），
而在交互式环境中阅读指南是更好地理解它们的好方法。

本教程和其他教程可能最方便在 Jupyter Notebook 中运行。

请参阅此处了解如何安装的说明。

#### 安装langchain 
!pip install langchain==0.2.1

#### 使用LangSmith
国内网络搞不了，我们就不搞这个了。


当然如果你可以，那你用吧。
主要就是用来看 langchin调用链，debug是真的方便。


您使用 LangChain 构建的许多应用程序将包含多个步骤，并多次调用 LLM 调用。随着这些应用程序变得越来越复杂，能够检查链或代理内部究竟发生了什么变得至关重要。最好的方法是使用 LangSmith。

在上面的链接处注册后，请确保设置环境变量以开始记录跟踪:


In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

### 使用语言模型
首先，让我们学习如何单独使用语言模型。LangChain 支持许多不同的语言模型，您可以互换使用 - 

在下面选择您想要使用的模型！ 选智谱就好了，其他你们想用就用其他的吧


In [ ]:
!pip install langchain_zhipu==4.1.5  # 怎么注册智谱官方怎么拿apikey，我这边就不多说了

In [2]:
import getpass
import os

os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

from langchain_zhipu import ChatZhipuAI

model = ChatZhipuAI(model="glm-3-turbo")

让我们首先直接使用该模型。
ChatZhipuAI 是 LangChain“Runnable”的实例，这意味着它们公开了一个用于与其交互的标准接口。
为了简单地调用该模型，我们可以将消息列表传递给 <code>.invoke </code>方法。

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="将以下内容从中文翻译成意大利语"),
    HumanMessage(content="你好！"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'id': '8705678841153504326', 'created': 1717085203, 'token_usage': {'completion_tokens': 5, 'prompt_tokens': 18, 'total_tokens': 23}, 'model_name': 'glm-3-turbo', 'finish_reason': 'stop'}, id='run-e0a6a26d-127b-47d2-b9fd-c495e59bb746-0')

如果我们启用了 LangSmith， 我们可以看到这次运行被记录到 LangSmith，并且可以看到 LangSmith 的跟踪 

### OutputParsers 输出解析器
请注意，模型的响应是 AIMessage。它包含字符串响应以及有关响应的其他元数据。

通常我们可能只想处理字符串响应。我们可以使用简单的输出解析器来解析此响应。

我们首先导入简单的输出解析器。

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

一种使用方法是直接使用它。例如，我们可以保存语言模型调用的结果，然后将其传递给解析器。

In [5]:
result = model.invoke(messages)

In [6]:
parser.invoke(result)

'Ciao!'

更常见的是，我们可以将模型与此输出解析器“链接”在一起。这意味着此输出解析器将在此链中每次被调用。

此链采用语言模型的输入类型（字符串或消息列表）并返回输出解析器的输出类型（字符串）。

我们可以使用 | 运算符轻松创建链。| 运算符在 LangChain 中用于将两个元素组合在一起。

In [7]:
chain = model | parser

In [8]:
chain.invoke(messages)

'Ciao!'

如果我们现在看看 LangSmith，我们可以看到该链有两个步骤：首先调用语言模型，然后将结果传递给输出解析器。我们可以看到 LangSmith 跟踪

### Prompt Templates  提示词模板

现在，我们将消息列表直接传递到语言模型中。这个消息列表从何而来？

通常，它由用户输入和应用程序逻辑组合而成。

此应用程序逻辑通常获取原始用户输入并将其转换为可传递给语言模型的消息列表。

常见的转换包括添加系统消息或使用用户输入格式化模板。

PromptTemplates 是 LangChain 中的一个概念，旨在协助进行这种转换。它们接收原始用户输入并返回可传递给语言模型的数据（提示）。

让我们在这里创建一个 PromptTemplate。它将接收两个用户变量：

`language`：要将文本翻译成的语言

`text`：要翻译的文本

In [9]:
# 先引入包
from langchain_core.prompts import ChatPromptTemplate

首先，让我们创建一个字符串，将其格式化为系统消息：

In [10]:
system_template = "将以下内容翻译成 {language}:"

接下来，我们可以创建 PromptTemplate。这将是 system_template 的组合，以及用于放置文本的更简单的模板

In [11]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

此提示模板的输入是个数组。我们可以单独试用此提示模板，看看它能不能翻译！

In [12]:
result = prompt_template.invoke({"language": "意大利", "text": "你好"})

result

ChatPromptValue(messages=[SystemMessage(content='将以下内容翻译成 意大利:'), HumanMessage(content='你好')])

我们可以看到它返回了一个由两条消息组成的 ChatPromptValue。如果我们想直接访问这些消息，我们可以这样做：

In [13]:
result.to_messages()

[SystemMessage(content='将以下内容翻译成 意大利:'), HumanMessage(content='你好')]

现在我们可以将其与上面的模型 和 输出解析器结合起来。这将把​​所有三个组件链接在一起。也就是LECL。

也就是将链路修改为： 提示词模板 -> 大模型调用 -> 整理输出的消息格式

In [14]:
chain = prompt_template | model | parser

In [15]:
chain.invoke({"language": "意大利", "text": "你好"})

'Ciao'

如果我们查看 LangSmith 跟踪，我们可以看到所有三个组件都出现在 LangSmith 跟踪中。

### 使用 LangServe 提供服务

现在我们已经构建了一个应用程序，我们需要为它提供服务。这就是 LangServe 的作用所在。

LangServe 帮助开发人员将 LangChain 链部署为 REST API。

您不需要使用 LangServe 来使用 LangChain，但在本指南中，我们将展示如何使用 LangServe 部署您的应用程序。

虽然本指南的第一部分旨在在 Jupyter Notebook 或脚本中运行，但现在我们将不再这样做。我们将创建一个 Python 文件，然后从命令行与其交互。

也就是使用langServe是不能用 Jupyter Notebook来运行了，要写个py文件，然后python运行。 

如果你熟悉fastapi就非常好理解了！

In [16]:
## 首先先安装这个langServe服务吧
!pip install "langserve[all]"

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple/
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.2 MB 1.6 MB/s eta 0:00:01
     ---- ----------------------------------- 0.1/1.2 MB 1.7 MB/s eta 0:00:01
     ---------- ----------------------------- 0.3/1.2 MB 2.4 MB/s eta 0:00:01
     --------------------- ------------------ 0.6/1.2 MB 3.6 MB/s eta 0:00:01
     ---------------------------------------  1.2/1.2 MB 5.3 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 4.9 MB/s eta 0:00:00


#### 编写py服务文件

要为我们的应用程序创建服务器，我们将创建一个 serve.py 文件。

它将包含为应用程序提供服务的逻辑。它由三部分组成：

我们刚刚在上面构建的链的定义

我们的 FastAPI 应用程序

为链提供服务的路由的定义，这是通过 `langserve.add_routes` 完成的

### 注意噢，下面那个是serve.py 文件的所有内容。 别在Jupyter Notebook 运行！会报错。

你应该新建一个serve.py ，让把下面的内容复制进去！

In [ ]:
# 注意噢，这个是serve.py 文件的所有内容。 千万别在Jupyter Notebook 运行！
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_zhipu import ChatZhipuAI
from langserve import add_routes
import getpass
import os

os.environ["ZHIPUAI_API_KEY"] = getpass.getpass()

# 1. Create prompt template
system_template = "将以下内容翻译成 {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatZhipuAI(model="glm-3-turbo")


# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)

怎么运行这个代码呢？   `python serve.py`

注意，conda环境要切换且在这个目录下运行即可。

然后我们打开 http://localhost:8000 就能看到啦！

### Playground

每个 LangServe 服务都带有一个简单的内置 UI，用于配置和调用应用程序，并提供流式输出和对中间步骤的可见性。

前往 http://localhost:8000/chain/playground/ 试用！

传入与之前相同的输入 - {"language": "意大利", "text": "你好"} - 它应该会像以前一样做出响应。

### Client 客户端调用

现在让我们设置一个客户端，以便以编程方式与我们的服务进行交互。

我们可以使用 [langserve.RemoteRunnable](/docs/langserve/#client) 轻松完成此操作。

使用它，我们可以与服务链进行交互，就像它在客户端运行一样。

In [ ]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

#### 结论
就是这样！在本教程中，我们介绍了如何创建第一个简单的 LLM 应用程序。

我们学习了如何使用语言模型、如何解析它们的输出、如何创建提示模板、如何将出色的可观察性融入使用 LangSmith 创建的链中以及如何使用 LangServe 部署它们。

这只是您成为熟练的 AI 工程师所需学习内容的冰山一角。幸运的是 - 我们还有很多其他资源！

有关更深入的教程，请查看我们的教程部分。

如果您对如何完成特定任务有具体问题，请参阅我们的操作指南部分。

要了解 LangChain 的核心概念，我们有详细的概念指南